<a href="https://colab.research.google.com/github/rohanmaladkar/AIML-journals/blob/main/_DE_SOLUTION_Rohan_maladkar_results_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 31 kB/s 
     |████████████████████████████████| 199 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=6d2e1c20d9f1586a151b3984f42f86b37407b5fd149a954bec6a3155098cc059
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

import numpy as np
import gdown
from pyspark.sql.window import Window
import pyspark.sql.functions as F


In [3]:
spark=SparkSession.builder.appName('WCh').getOrCreate()

In [4]:
spark

In [5]:
df_pyspark_game = spark.read.option('header','true').csv('/content/drive/MyDrive/data_grokr/chess_wc_history_game_info.csv',inferSchema=True)
df_pyspark_moves = spark.read.option('header','true').csv('/content/drive/MyDrive/data_grokr/chess_wc_history_moves.csv',inferSchema=True)
df_pyspark_eco = spark.read.option('header','true').csv('/content/drive/MyDrive/data_grokr/eco_codes.csv',inferSchema=True)

In [6]:
df_pyspark_game.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+-------------------+---------------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|       date_created|tournament_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+-------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw| 

In [7]:
  df_pyspark_game.printSchema()

root
 |-- game_id: string (nullable = true)
 |-- game_order: integer (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: double (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: integer (nullable = true)
 |-- black_elo: integer (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: integer (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: integer (nullable = true)
 |-- winner_loser_elo_diff: integer (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: timestamp (nullable = true)
 |-- tournament_name: string (nullable = true)



In [8]:
df_pyspark_moves.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [9]:
df_pyspark_eco.show()

+---+--------------------+--------------------+--------+---------+
|eco|            eco_name|         eco_example|eco_type|eco_group|
+---+--------------------+--------------------+--------+---------+
|A00|    Uncommon Opening|  1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|                1 b3|       A|    Flank|
|A02|      Bird's Opening|                1 f4|       A|    Flank|
|A03|      Bird's Opening|             1 f4 d5|       A|    Flank|
|A04|        Reti Opening|               1 Nf3|       A|    Flank|
|A05|        Reti Opening|           1 Nf3 Nf6|       A|    Flank|
|A06|        Reti Opening|            1 Nf3 d5|       A|    Flank|
|A07|King's Indian Attack|       1 Nf3 d5 2 g3|       A|    Flank|
|A08|King's Indian Attack|1 Nf3 d5 2 g3 c5 ...|       A|    Flank|
|A09|        Reti Opening|       1 Nf3 d5 2 c4|       A|    Flank|
|A10|             English|                1 c4|       A|    Flank|
|A11|English, Caro-Kan...|             1 c4 c6|       A|    Fl

In [13]:
print(" df_pyspark_game","rows:",df_pyspark_game.count(),"columns:",len(df_pyspark_game.columns))
print("df_pyspark_moves","rows:",df_pyspark_moves.count(),"columns:",len(df_pyspark_moves.columns))
print("df_pyspark_eco","rows:",df_pyspark_eco.count(),"columns:",len(df_pyspark_eco.columns))

 df_pyspark_game rows: 2938 columns: 21
df_pyspark_moves rows: 252948 columns: 64
df_pyspark_eco rows: 500 columns: 5


# ** 1. List of Winners of Each World champions Trophy**

In [10]:
df_pyspark_game.select(['winner','tournament_name']).show()
#df_pyspark.select(['winner','tournament_name']).show(2939)

+---------+---------------+
|   winner|tournament_name|
+---------+---------------+
|     draw| WorldChamp2021|
|     draw| WorldChamp2021|
|     draw| WorldChamp2021|
|     draw| WorldChamp2021|
|     draw| WorldChamp2021|
|Carlsen,M| WorldChamp2021|
|     draw| WorldChamp2021|
|Carlsen,M| WorldChamp2021|
|Carlsen,M| WorldChamp2021|
|     draw| WorldChamp2021|
|Carlsen,M| WorldChamp2021|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
|     draw| WorldChamp2018|
+---------+---------------+
only showing top 20 rows



In [11]:
var1 = df_pyspark_game.filter((df_pyspark_game.tournament_name.contains('WorldChamp')) & (~df_pyspark_game['winner'].isin(['draw'])))
var1 = var1.groupBy("tournament_name", "winner").count().orderBy("tournament_name")
win = Window.partitionBy('tournament_name').orderBy(F.col('count').desc())
var1 = var1.withColumn('row', F.row_number().over(win)).select("winner", "tournament_name","count","row").where(F.col("row")==1)
var1=var1.drop(*["count","row"])
var1.show()

+--------------------+---------------+
|              winner|tournament_name|
+--------------------+---------------+
|   Steinitz, William| WorldChamp1886|
|   Steinitz, William| WorldChamp1889|
|   Steinitz, William| WorldChamp1890|
|   Steinitz, William| WorldChamp1892|
|     Lasker, Emanuel| WorldChamp1894|
|     Lasker, Emanuel| WorldChamp1896|
|     Lasker, Emanuel| WorldChamp1907|
|     Lasker, Emanuel| WorldChamp1908|
|     Lasker, Emanuel| WorldChamp1909|
|    Schlechter, Carl|WorldChamp1910a|
|     Lasker, Emanuel|WorldChamp1910b|
|Capablanca, Jose ...| WorldChamp1921|
| Alekhine, Alexander| WorldChamp1927|
| Alekhine, Alexander| WorldChamp1929|
| Alekhine, Alexander| WorldChamp1934|
|           Euwe, Max| WorldChamp1935|
| Alekhine, Alexander| WorldChamp1937|
|  Botvinnik, Mikhail| WorldChamp1948|
|  Botvinnik, Mikhail| WorldChamp1951|
|  Botvinnik, Mikhail| WorldChamp1954|
+--------------------+---------------+
only showing top 20 rows



In [10]:
#df_pyspark_moves.withColumnRenamed('player','player_name').show()

In [18]:

#df_pyspark = df_pyspark_moves.withColumn('number_of_wins',df_pyspark_moves['winner'])

In [13]:
training = spark.read.csv('/content/drive/MyDrive/data_grokr/chess_wc_history_game_info.csv',header=True,inferSchema=True)

In [20]:
training.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+-------------------+---------------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|       date_created|tournament_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+-------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw| 

In [21]:
training.columns

['game_id',
 'game_order',
 'event',
 'site',
 'date_played',
 'round',
 'white',
 'black',
 'result',
 'white_elo',
 'black_elo',
 'white_title',
 'black_title',
 'winner',
 'winner_elo',
 'loser',
 'loser_elo',
 'winner_loser_elo_diff',
 'eco',
 'date_created',
 'tournament_name']

In [23]:
# def str_lables(a):
#   if a==1:
#     return 'white'
#   elif a==0:
#     return 'black'
  

In [24]:
# from pyspark.ml.feature import VectorAssembler
# featureassembler=VectorAssembler(inputCols=["white","black"],outputCol="player_name")

In [26]:
#output=featureassembler.transform(training)

**2. List of Players with number of times they have won Tournament in descending order**

In [17]:
from pyspark.ml.feature import StringIndexer

In [19]:
indexer = StringIndexer(inputCols=["white","black","winner"],outputCols=["white_index","black_index","winner_index"])
df_r = indexer.fit(df_pyspark_game).transform(df_pyspark_game)
df_r.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+-------------------+---------------+-----------+-----------+------------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|       date_created|tournament_name|white_index|black_index|winner_index|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+-------------------+---------------+-----------+-----------+------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi

In [22]:
from pyspark.ml.feature import VectorAssembler
coef_var = ["white_index","black_index"]
featureassembler=VectorAssembler(inputCols=coef_var,outputCol="player_name")
output = featureassembler.transform(df_r)
output.select('player_name').show()

+-------------+
|  player_name|
+-------------+
| [133.0,85.0]|
|[107.0,159.0]|
| [133.0,85.0]|
|[107.0,159.0]|
| [133.0,85.0]|
|[107.0,159.0]|
| [133.0,85.0]|
|[107.0,159.0]|
| [133.0,85.0]|
|[107.0,159.0]|
| [133.0,85.0]|
| [125.0,24.0]|
| [22.0,126.0]|
| [125.0,24.0]|
| [22.0,126.0]|
| [125.0,24.0]|
| [22.0,126.0]|
| [22.0,126.0]|
| [125.0,24.0]|
| [22.0,126.0]|
+-------------+
only showing top 20 rows



In [33]:
var2 = df_pyspark_game.filter((df_pyspark_game.winner != 'draw')& (~df_pyspark_game.event.contains('KO')) & (~df_pyspark_game.event.contains('k.o')))



In [34]:
var2 = var2.groupBy("tournament_name", "winner").count()


In [35]:
win2 = Window.partitionBy('tournament_name').orderBy(F.col('count').desc())


In [36]:
var2 = var2.withColumn('row', F.row_number().over(win2)).where(F.col('row') == 1)


In [37]:
var2 = var2.drop(*["count","row","tournament_name"])


In [38]:
var2 = var2.groupBy("winner").count().orderBy(F.col('count').desc())


In [39]:
var2 = var2.withColumnRenamed("winner", "player_name")
var2 = var2.withColumnRenamed("count", "number_of_wins")

In [13]:
var2.show()

+--------------------+--------------+
|         player_name|number_of_wins|
+--------------------+--------------+
|     Lasker, Emanuel|             6|
|      Kasparov, Gary|             6|
|     Karpov, Anatoly|             5|
|  Botvinnik, Mikhail|             5|
|   Steinitz, William|             4|
| Alekhine, Alexander|             4|
|           Kramnik,V|             3|
|     Carlsen, Magnus|             3|
|             Anand,V|             3|
| Petrosian, Tigran V|             2|
|           Carlsen,M|             1|
|    Smyslov, Vassily|             1|
|           Topalov,V|             1|
|    Schlechter, Carl|             1|
|           Euwe, Max|             1|
|    Spassky, Boris V|             1|
|Fischer, Robert J...|             1|
|         Tal, Mihail|             1|
|Capablanca, Jose ...|             1|
|           Gelfand,B|             1|
+--------------------+--------------+



In [23]:
final_result = output.select("player_name","winner_index")

In [36]:
# from pyspark.ml.regression import LinearRegression
# train_data,test_data = final_result.randomSplit([0.75,0.25])
# regressor = LinearRegression(featuresCol='player_name', labelCol='winner_index')
# model = regressor.fit(train_data)

In [24]:
#pd.DataFrame({"coefficients":model.coefficients}, index=coef_var)

In [24]:
# res = model.evaluate(test_data)
# res

In [30]:
# unlabled = test_data.select('player_name','winner_index')

In [31]:
# pred = model.transform(unlabled)

# **3. Most and Least Popular eco move in world championship history**

In [51]:
from pyspark.sql.functions import greatest

In [55]:
df_pyspark_eco.show()

+---+--------------------+--------------------+--------+---------+
|eco|            eco_name|         eco_example|eco_type|eco_group|
+---+--------------------+--------------------+--------+---------+
|A00|    Uncommon Opening|  1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|                1 b3|       A|    Flank|
|A02|      Bird's Opening|                1 f4|       A|    Flank|
|A03|      Bird's Opening|             1 f4 d5|       A|    Flank|
|A04|        Reti Opening|               1 Nf3|       A|    Flank|
|A05|        Reti Opening|           1 Nf3 Nf6|       A|    Flank|
|A06|        Reti Opening|            1 Nf3 d5|       A|    Flank|
|A07|King's Indian Attack|       1 Nf3 d5 2 g3|       A|    Flank|
|A08|King's Indian Attack|1 Nf3 d5 2 g3 c5 ...|       A|    Flank|
|A09|        Reti Opening|       1 Nf3 d5 2 c4|       A|    Flank|
|A10|             English|                1 c4|       A|    Flank|
|A11|English, Caro-Kan...|             1 c4 c6|       A|    Fl

In [26]:
var3 = df_pyspark_game.filter(df_pyspark_game.tournament_name.contains('WorldChamp'))



In [27]:
var3 = var3.groupBy("eco").count().orderBy(F.col('count').desc())

In [28]:
var3 = var3.head(1) + var3.tail(1)


In [29]:
var3 = spark.createDataFrame(var3)


In [30]:
var3 = var3.join(df_pyspark_eco, on='eco', how="inner")


In [31]:
var3 = var3.withColumnRenamed("count", "number_of_occurences")
var3 = var3.select("eco", "eco_name", "number_of_occurences").orderBy(F.col("number_of_occurences"))


In [32]:
var3.show()

+---+------------------+--------------------+
|eco|          eco_name|number_of_occurences|
+---+------------------+--------------------+
|A44|Old Benoni Defense|                   1|
|C42|    Petrov Defense|                  21|
+---+------------------+--------------------+



# **4. Find the eco move with most winnings.**

In [40]:
var4 = df_pyspark_game.filter(df_pyspark_game.winner != "draw")


In [41]:
var4 = var4.groupby("eco").count()


In [42]:
var4 = var4.orderBy(F.col("count").desc())


In [43]:
var4 = var4.head(1)


In [44]:
var4 = spark.createDataFrame(var4)


In [45]:
var4 = var4.join(df_pyspark_eco, on="eco", how="inner").select("eco", "eco_name")


In [46]:
var4.show()

+---+-----------------+
|eco|         eco_name|
+---+-----------------+
|B90|Sicilian, Najdorf|
+---+-----------------+



# **5. Longest and shortest game ever played in a world championship in terms of move.**

In [49]:
win5 = Window.partitionBy("game_id").orderBy(F.col('move_no_pair').desc())


In [50]:
var5 = df_pyspark_moves.withColumn('row', F.row_number().over(win5)).select("game_id", "move_no_pair")


In [51]:
var5 = var5.filter(F.col("row")==1)


In [52]:
var5 = var5.join(df_pyspark_game, on='game_id', how='inner').select("game_id","event","tournament_name","move_no_pair").orderBy(F.col('move_no_pair').desc())


In [53]:
var5 = var5.filter((df_pyspark_game.tournament_name.contains('WorldChamp')))


In [54]:
var5 = var5.withColumnRenamed("move_no_pair", "number_of_moves")


In [55]:
var5 = var5.head(1) + var5.tail(1)
var5 = spark.createDataFrame(var5)


In [56]:
var5.show()

+--------------------+--------------------+---------------+---------------+
|             game_id|               event|tournament_name|number_of_moves|
+--------------------+--------------------+---------------+---------------+
|58e83255-93bb-4d5...|            WCh 2021| WorldChamp2021|            136|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|              1|
+--------------------+--------------------+---------------+---------------+



# **6. Shortest and Longest Draw game ever Played.**

In [62]:
win6 = Window.partitionBy("game_id").orderBy(F.col('move_no_pair').desc())


In [63]:
var6 = df_pyspark_moves.withColumn('row', F.row_number().over(win6)).select("game_id", "move_no_pair")


In [64]:
var6 = var6.filter(F.col("row")==1)


In [65]:
var6 = var6.join(df_pyspark_game, on='game_id', how='inner').select("game_id","event","tournament_name","move_no_pair").orderBy(F.col('move_no_pair').desc())


In [66]:
var6 = var6.filter((df_pyspark_game.winner.contains('draw')))
var6 = var6.withColumnRenamed("move_no_pair", "number_of_moves")


In [67]:
var6 = var6.head(1) + var6.tail(1)
var6 = spark.createDataFrame(var6)
var6.show()

+--------------------+-------------+---------------+---------------+
|             game_id|        event|tournament_name|number_of_moves|
+--------------------+-------------+---------------+---------------+
|88f34084-e4df-490...|  FIDE WCh KO|  FideChamp2002|            129|
|a27c2b95-fb62-4b6...|FIDE-Wch k.o.|  FideChamp1998|              6|
+--------------------+-------------+---------------+---------------+



# **7. Most and Least rated Player.**

In [68]:
A7white = df_pyspark_game.filter(df_pyspark_game.white_elo.isNotNull())
A7white = A7white.groupBy("white").max("white_elo")
A7white = A7white.withColumnRenamed("white","player_name")


In [69]:
A7black = df_pyspark_game.filter(df_pyspark_game.white_elo.isNotNull())
A7black = A7black.groupBy("black").max("black_elo")
A7black = A7black.withColumnRenamed("black","player_name")

In [70]:
var7 = A7white.join(A7black, on='player_name', how='inner')


In [71]:
var7 = var7.withColumn("elo",(F.col("max(white_elo)")+F.col("max(black_elo)"))/2)


In [72]:
var7 = var7.drop( *["max(white_elo)", "max(black_elo)"])
var7 = var7.orderBy( F.col("elo").desc())


In [73]:
var7 = var7.head(1) + var7.tail(1)
var7 = spark.createDataFrame(var7)
var7.show()

+---------------+------+
|    player_name|   elo|
+---------------+------+
|Carlsen, Magnus|2870.0|
|      Abulhul,T|2076.0|
+---------------+------+



# **8. 3rd Last Player with most Loss.**

In [74]:
var8 = df_pyspark_game.filter(~df_pyspark_game['winner'].isin(['draw']))


In [75]:
var8 = var8.groupBy("loser").count()


In [76]:
var8 = var8.withColumnRenamed("loser","player_name")
var8 = var8.orderBy(F.col("count").desc())


In [77]:
win8 = Window.orderBy(F.col("count").desc())


In [78]:
var8 = var8.withColumn("index", F.row_number().over(win8))


In [79]:
var8 = var8.filter((var8['index']==3))
var8 = var8.drop(*["count","index"])


In [80]:
var8.show()

+---------------+
|    player_name|
+---------------+
|Karpov, Anatoly|
+---------------+



# **9. How many times players with low rating won matches with their total win Count**

In [81]:
var9 = df_pyspark_game.filter(df_pyspark_game["winner_loser_elo_diff"]<0)


In [82]:
var9 = var9.groupBy("winner").count().orderBy(F.col('count').desc())


In [83]:
var9 = var9.withColumnRenamed("winner","player_name")
var9 = var9.withColumnRenamed("count","win_count")


In [84]:
var9.show()

+--------------------+---------+
|         player_name|win_count|
+--------------------+---------+
|     Karpov, Anatoly|       20|
|      Kasimdzhanov,R|       11|
|           Macieja,B|        9|
|  Kortschnoj, Viktor|        8|
|           Kramnik,V|        8|
|          Grischuk,A|        7|
|        Ponomariov,R|        7|
|Khalifman, Alexander|        6|
|      Short, Nigel D|        5|
|   Akopian, Vladimir|        5|
|      Kasparov, Gary|        5|
|           Sulskis,S|        5|
|         Dominguez,L|        5|
|           Ehlvest,J|        5|
|Nisipeanu, Liviu ...|        4|
| Tkachiev, Vladislav|        4|
|          Nakamura,H|        4|
|           Gelfand,B|        4|
|         Filippov,Va|        4|
|              Ni Hua|        4|
+--------------------+---------+
only showing top 20 rows



# **10. Move Sequence for Each Player in a Match**

In [15]:
win10 = Window.partitionBy("game_id").orderBy(F.col("move_no").desc())


In [16]:
var10 = df_pyspark_moves.withColumn("row",F.row_number().over(win10))


In [17]:
var10 = var10.filter(var10.row<3)


In [18]:
var10 = var10.select("game_id", "player", "move_sequence", "move_no")
var10 = var10.withColumnRenamed("move_no", "move_count")
var10 = var10.withColumnRenamed("player", "player_name")


In [19]:
var10.show()

+--------------------+--------------------+--------------------+----------+
|             game_id|         player_name|       move_sequence|move_count|
+--------------------+--------------------+--------------------+----------+
|00563036-1aa1-42c...|Azmaiparashvili, ...|e4|c5|c3|Nf6|e5|N...|        89|
|00563036-1aa1-42c...|   Krasenkow, Michal|e4|c5|c3|Nf6|e5|N...|        88|
|007ba523-8617-42b...|Fischer, Robert J...|e4|Nf6|e5|Nd5|d4|...|       148|
|007ba523-8617-42b...|    Spassky, Boris V|e4|Nf6|e5|Nd5|d4|...|       147|
|00c5cb30-540b-47b...|    Slobodjan, Roman|d4|d5|c4|e6|Nf3|c...|        58|
|00c5cb30-540b-47b...|Azmaiparashvili, ...|d4|d5|c4|e6|Nf3|c...|        57|
|00ff8f8a-33d1-466...| Tkachiev, Vladislav|d4|Nf6|Nf3|e6|c4|...|        49|
|00ff8f8a-33d1-466...|  Hoffman, Alejandro|d4|Nf6|Nf3|e6|c4|...|        48|
|0128c0c8-d901-4c8...|      Kasparov, Gary|c4|c5|Nf3|Nf6|Nc3...|       114|
|0128c0c8-d901-4c8...|     Karpov, Anatoly|c4|c5|Nf3|Nf6|Nc3...|       113|
|01af512b-f6

# 11. Total Number of games where losing player has more Captured score than Winning **player**

In [20]:
var11 = df_pyspark_game.where(~df_pyspark_game['winner'].isin(['draw']))
var11 = var11.join(df_pyspark_moves, on="game_id", how="inner")
win11 = Window.partitionBy("game_id").orderBy(F.col("move_no").desc())

In [21]:
var11 = var11.withColumn("row", F.row_number().over(win11))
var11 = var11.filter(var11.row == 1).drop("row")

In [22]:
var11 = var11.withColumn("loser_white",(var11['loser']==var11['white']))
var11 = var11.withColumn("loser_score",F.when(F.col("loser_white"),F.col("captured_score_for_white")).otherwise(( F.col('captured_score_for_black'))))


In [23]:
var11 = var11.withColumn("winner_white",(var11['winner']==var11['white']))
var11 = var11.withColumn("winner_score",F.when(F.col("winner_white"),F.col("captured_score_for_white")).otherwise(( F.col('captured_score_for_black'))))

In [24]:
var11 = var11.withColumn("Loser_with_higher_core",var11["loser_score"]>var11["winner_score"])
var11 = var11.filter(var11["Loser_with_higher_core"]==True) 

In [25]:
finalwinA11 = Window.partitionBy().orderBy(F.col('game_id'))

In [26]:
var11 = var11.withColumn( "total_number_of_games", F.row_number().over(finalwinA11) )
var11 = var11.where( var11.total_number_of_games == var11.count() )
var11 = var11.select("total_number_of_games")

In [27]:
var11.show()

+---------------------+
|total_number_of_games|
+---------------------+
|                  147|
+---------------------+



# **12. List All Perfect Tournament with Winner Name**

In [46]:
var12 = df_pyspark_game.filter((~df_pyspark_game.winner.isin(['draw'])) & (~df_pyspark_game.event.contains('KO')) & (~df_pyspark_game.event.contains('k.o')))

In [47]:
wvar12 = Window.partitionBy("tournament_name")

In [49]:
var12 = var12.withColumn( "winner_all_tournament", F.size( F.collect_set('winner').over(wvar12) ) == 1 )


In [50]:
var12 = var12.where( F.col("winner_all_tournament") )
var12 = var12.groupBy("tournament_name", "winner").count().orderBy("tournament_name")
var12= var12.withColumnRenamed("winner", "winner_name")
var12 = var12.select("winner_name", "tournament_name")
var12.show()

+--------------------+---------------+
|         winner_name|tournament_name|
+--------------------+---------------+
|     Lasker, Emanuel| WorldChamp1907|
|     Lasker, Emanuel|WorldChamp1910b|
|Capablanca, Jose ...| WorldChamp1921|
|           Kramnik,V| WorldChamp2000|
|     Carlsen, Magnus| WorldChamp2013|
|           Carlsen,M| WorldChamp2021|
+--------------------+---------------+



# **13. Player with highest winning ratio**

In [51]:
var13_data = df_pyspark_game.filter(~df_pyspark_game.winner.isin(['draw']))
var13played = var13_data.select( F.explode( F.array("white", "black" ) ).alias("player"))
var13played = var13played.groupBy("player").agg(F.count("player").alias("Games_played"))


In [52]:
var13won = var13_data.groupBy("winner").agg(F.count("winner").alias("Games_won"))
var13won = var13won.withColumnRenamed("winner","player")

In [53]:
var13ratio = var13played.join(var13won,on= "player", how="inner")
var13ratio = var13ratio.withColumn("Win_ratio",F.col("Games_won")/F.col("Games_played"))
var13ratio = var13ratio.orderBy(F.col("Win_ratio").desc())


In [54]:
var13 = var13ratio.head(1)
var13 = spark.createDataFrame(var13)
var13 = var13.select("player")
var13 = var13.withColumnRenamed("player", "player_name")

In [55]:
var13.show()

+--------------------+
|         player_name|
+--------------------+
|Stepovaia Dianche...|
+--------------------+



# **14. Player who had given checkmate with Pawn**

In [58]:
var14 = df_pyspark_moves.filter((df_pyspark_moves.is_check_mate == 1) & df_pyspark_moves.piece.contains('P'))



In [59]:
var14 = var14.select('player')
var14 = var14.withColumnRenamed("player", "player_name")


In [60]:
var14.show()

+--------------+
|   player_name|
+--------------+
|Andersson, Ulf|
+--------------+



# **15. List games where player has won game without queen**

In [61]:
var15 = df_pyspark_moves.filter(((df_pyspark_moves['white_queen_count']==0)|(df_pyspark_moves["black_queen_count"]==0)))


In [62]:
var15 = var15.join(df_pyspark_game, on='game_id', how="inner")
var15 = var15.filter(df_pyspark_game.winner != 'draw')


In [63]:
var15 = var15.groupBy("game_id","event","winner").count().orderBy("game_id")
win15 = Window.partitionBy("game_id").orderBy(F.col("count").desc())


In [64]:
var15 = var15.withColumn('row', F.row_number().over(win15)).where(F.col("row")==1)
var15 = var15.select("game_id","event","winner").distinct()


In [67]:
var15 = var15.drop(*["count","row"])
var15 = var15.withColumnRenamed("winner","player_name")
var15.show()

+--------------------+--------------------+--------------------+
|             game_id|               event|         player_name|
+--------------------+--------------------+--------------------+
|007ba523-8617-42b...|World Championshi...|Fischer, Robert J...|
|00bdc5cb-22b3-4fc...|FIDE-Wch k.o. g/2...|       Polgar, Judit|
|01102870-dbe8-4fd...|         FIDE WCh KO|         Kotronias,V|
|0128c0c8-d901-4c8...|World Championshi...|     Karpov, Anatoly|
|01af512b-f6cd-471...|World Championshi...|    Bogoljubow, Efim|
|0230130d-ee51-4f9...|       FIDE-Wch k.o.|     Barua, Dibyendu|
|02797e38-3fa8-46f...|FIDE-Wch k.o. g/2...|       Dreev, Alexey|
|02d27637-16cd-4d9...|World Championshi...|           Euwe, Max|
|02ddab74-4fff-47b...|       FIDE-Wch k.o.|        Milov, Vadim|
|03098c35-6489-4b4...|FIDE-Wch k.o. g/2...|   Kramnik, Vladimir|
|03ad168a-2860-4e8...|         FIDE WCh KO|            Shirov,A|
|042f3677-18cb-48d...|       FIDE-Wch k.o.|    Seirawan, Yasser|
|04565968-1b5a-4af...|   